In [1]:
import tarfile
import scipy.io
import numpy as np
import os
import cv2 as cv
import shutil
import random
from console_progressbar import ProgressBar #untuk proses loading data


In [7]:
def ensure_folder(folder): #untuk membuat folder baru hasil crop sesuai dengan class
    if not os.path.exists(folder):
        os.makedirs(folder)
        
def save_train_data(fnames, labels, bboxes):
    src_folder ='cars_train' #alamat data train
    num_samples = len(fnames) #banyaknya data train

    train_split = 0.8 #train 80%, validasi 20%
    num_train = int(round(num_samples * train_split))
    train_indexes = random.sample(range(num_samples), num_train) #mengambil index random untuk training

    pb = ProgressBar(total=100, prefix='Save train data', suffix='', decimals=3, length=50, fill='=') #untuk loadingg data

    for i in range(num_samples):
        fname = fnames[i]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i] #mengambil data koordinat dari dict .math

        src_path = os.path.join(src_folder, fname) #menamakan gambar
        src_image = cv.imread(src_path) #read gambar
        height, width = src_image.shape[:2] #mengambil tinggi dan lebar gambar
        # margins of 16 pixels
        margin = 16 #untuk treshold
        x1 = max(0, x1 - margin) 
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height) #mencari data koordinat untuk proses cropping
        # print("{} -> {}".format(fname, label))
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        if i in train_indexes:
            dst_folder = 'kaggle/working/data/train/' #menuju folder apabila termasuk data train
        else:
            dst_folder = 'kaggle/working/data/valid/' #menuju folder apabila termasuk data validasi

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path) #jika ada foldernya, maka taruh filenya disitu
        dst_path = os.path.join(dst_path, fname)

        crop_image = src_image[y1:y2, x1:x2] #crop filenya
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)


# In[4]:


def save_test_data(fnames, bboxes): #sama penjelasannya
    src_folder = 'cars_test'
    dst_folder = 'kaggle/working/data/test/'
    num_samples = len(fnames)

    pb = ProgressBar(total=100, prefix='Save test data', suffix='', decimals=3, length=50, fill='=')

    for i in range(num_samples):
        fname = fnames[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        # print(fname)
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        dst_path = os.path.join(dst_folder, fname)
        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)


# In[5]:


def process_train_data():
    print("Processing train data...")
    cars_annos = scipy.io.loadmat('cars_train_annos.mat') #data koordinat,class dan nama
    annotations = cars_annos['annotations'] #ambil anotasinya saja dalam array
    annotations = np.transpose(annotations) #transpose supaya data menurun

    fnames = []
    class_ids = []
    bboxes = []
    labels = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0] 
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        class_id = annotation[0][4][0][0] #akses kolom2nya
        labels.append('%04d' % (class_id,))
        fname = annotation[0][5][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        class_ids.append(class_id)
        fnames.append(fname)

    labels_count = np.unique(class_ids).shape[0] #mencari unique kelas (ada 196 kelas)
    print(np.unique(class_ids))
    print('The number of different cars is %d' % labels_count)

    save_train_data(fnames, labels, bboxes) #menuju fungsi save_train_data untuk save data yang telah dicrop


# In[6]:


def process_test_data(): #sama penjelasannya
    print("Processing test data...")
    cars_annos = scipy.io.loadmat('cars_test_annos.mat')
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    bboxes = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        fname = annotation[0][4][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        fnames.append(fname)

    save_test_data(fnames, bboxes)


# In[7]:


img_width, img_height = 299, 299 #image diseragamkan menjadi ukuran berikut

cars_meta = scipy.io.loadmat('cars_meta.mat') #load isi nama class
class_names = cars_meta['class_names']  # shape=(1, 196)
class_names = np.transpose(class_names)
print('class_names.shape: ' + str(class_names.shape))
print('Sample class_name: [{}]'.format(class_names[8][0][0]))

ensure_folder('kaggle/working/data/train') #menuju fungsi ensure_folder untuk membuat folder tsb (ketika blm ada)
ensure_folder('kaggle/working/data/valid')
ensure_folder('kaggle/working/data/test')

process_train_data() #menuju modul tsb
process_test_data() #menuju modul tsb

class_names.shape: (196, 1)
Sample class_name: [Aston Martin V8 Vantage Coupe 2012]
Processing train data...
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196]
The number of different cars is 196
Save train data |==========================================

# ----------------------DATA PREPROCESSING---------------------------
# Nama Kelompok:
## Rizki, Rando, Sayyid, Ilfani